In [1]:
import zipfile,fnmatch,os
import zipfile,fnmatch,os
import pandas as pd
import pickle
from pathlib import Path



In [2]:
rootPath = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD-20220114T103719Z-002\\NSE_Equity_Futures_iEOD'
pattern = '*BANKNIFTY.txt'
def unzip_folders(rootPath,pattern):
    for root, dirs, files in os.walk(rootPath):
        for filename in fnmatch.filter(files, pattern):
            print(os.path.join(root, filename))
            zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(root, os.path.splitext(filename)[0]))
            os.remove(os.path.join(root, filename))

In [3]:
def convert_df_to_timeseries(df):
    df['date_time'] = df['date'].astype(str) + ' ' + df['time']
    df = df.sort_values(by='date_time')
    df.index = df['date_time']
    df = df[['open','high','low','close']]
    return df

In [4]:
def convert_df_to_timeseries(df):
    df['date_time'] = df['date'].astype(str) + ' ' + df['time']
    df = df.sort_values(by='date_time')
    df.index = df['date_time']
    df = df[['open','high','low','close']]
    return df

def create_dataset(root_path,pattern,data_save_path,data_name,reset_df = False):
    files_list = []
    bad_files = []
    files_processed = []
    base_df = pd.DataFrame(columns = ['name','date','time','open','high','low','close'])
    if not os.path.exists(f'{data_save_path}{data_name}/'):
        os.makedirs(f'{data_save_path}{data_name}/')
        print(f'Created folder {data_save_path}{data_name}')

    already_loaded_file_name = f'{data_save_path}{data_name}/already_loaded_files.pickle'
    csv_save_location = f'{data_save_path}{data_name}/{data_name}.csv'
    print(f'Data save path is {csv_save_location}')
    print(f'File with already loaded files is {already_loaded_file_name}')
    orig_cols = ['name','date','time','open','high','low','close']
    try:
        with open(already_loaded_file_name, 'rb') as handle:
            already_loaded_files = pickle.load(handle)
            already_loaded_files = [Path(col) for col in already_loaded_files]
            print(f"Total files already saved {len(already_loaded_files)}")
    except Exception as e1:
        print(f"File {already_loaded_file_name} is not loaded because of error : {e1}")
        already_loaded_files = []
    for root, dirs, files in os.walk(root_path):
        for filename in fnmatch.filter(files, pattern):
            f_name = Path(os.path.join(root, filename))
            files_list.append(f_name)
    files_to_be_loaded = [f for f in files_list if f not in already_loaded_files]
    files_to_be_loaded = list(dict.fromkeys(files_to_be_loaded))
    files_list = list(dict.fromkeys(files_list))
    print(f"Total files detected {len(files_list)}")
    print(f"Total new files detected {len(files_to_be_loaded)}")
    try:
        base_df = pd.read_csv(csv_save_location)
    except Exception as e1:
        print(f"Error while loading dataframe from {csv_save_location} because of error : {e1}")
        base_df = pd.DataFrame(columns = ['open','high','low','close'])
        files_to_be_loaded = files_list
    if len(base_df) == 0 or reset_df:
        files_to_be_loaded = files_list
        print(f"We are going to reload all the data")

    print(f"Number of files to be loaded {len(files_to_be_loaded)}")
    base_df_st_shape = base_df.shape
    for i,f_name in enumerate(files_to_be_loaded,1):
        f_name = os.path.join(root, f_name)
        try:
            tmp_df = pd.read_csv(f_name,header=None)
            tmp_df = tmp_df.loc[:,0:6]
            tmp_df.columns = orig_cols
            tmp_df = convert_df_to_timeseries(tmp_df)
            base_df = pd.concat([base_df,tmp_df],axis=0)
            print(len(files_to_be_loaded)-i,base_df.shape,f_name)
            already_loaded_files.append(f_name)
        except Exception as e1:
            bad_files.append(f_name)
            print(f"File {f_name} is not loaded because of error : {e1}")
    with open(already_loaded_file_name, 'wb') as handle:
        pickle.dump(already_loaded_files, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(f"Shape of the dataframe before duplicate drop is {base_df.shape}")
    base_df = base_df.drop_duplicates()
    print(f"Shape of the dataframe after duplicate drop is {base_df.shape}")
    if base_df_st_shape != base_df.shape:
        base_df = base_df.sort_index()
        base_df.to_csv(csv_save_location, index_label=False )
        print(f"Saving dataframe to location {csv_save_location}")
    return base_df


In [5]:
data_save_path = './data/'
root_path = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD\\'
pattern = '*BANKNIFTY.txt'
data_name = 'banknifty'
base_df= create_dataset(root_path,pattern,data_save_path,data_name)

Data save path is ./data/banknifty/banknifty.csv
File with already loaded files is ./data/banknifty/already_loaded_files.pickle
Total files already saved 1279
Total files detected 1280
Total new files detected 1
Number of files to be loaded 1
File C:\Users\8prab\Documents\NSE_Equity_Futures_iEOD\2018\AUG\29AUG\29AUG\BANKNIFTY.txt is not loaded because of error : Length mismatch: Expected axis has 1 elements, new values have 7 elements
Shape of the dataframe before duplicate drop is (1170729, 4)
Shape of the dataframe after duplicate drop is (1170729, 4)


In [6]:
def label_generator(val):
    if val <= 10 and val>=10:
        return '-10to10'
    elif val > 10 and val <= 20:
        return '10to20'
    elif val > 20 and val <= 40:
        return '20to40'
    elif val > 40 and val <= 60:
        return '40to60'
    elif val > 60 and val <= 80:
        return '60to80'
    elif val > 80 and val <= 100:
        return '80to100'
    elif val > 100:
        return 'above100'
    elif val < -10 and val >= -20:
        return '-10to-20'
    elif val < -20 and val >= 40:
        return '-20to-40'
    elif val < -40 and val >= 60:
        return '-40to-60'
    elif val < -60 and val >= 80:
        return '-60to-80'
    elif val < -80 and val >= 100:
        return '-80to-100'
    elif val < 100:
        return 'below100'
    else:
        return 'unknown'

In [ ]:
def encode_percentage_change(val):
    if val <= 10 and val>=10:
        return '-10to10'
    elif val > 10 and val <= 20:
        return '10to20'
    elif val > 20 and val <= 40:
        return '20to40'
    elif val > 40 and val <= 60:
        return '40to60'
    elif val > 60 and val <= 80:
        return '60to80'
    elif val > 80 and val <= 100:
        return '80to100'
    elif val > 100:
        return 'above100'
    elif val < -10 and val >= -20:
        return '-10to-20'
    elif val < -20 and val >= 40:
        return '-20to-40'
    elif val < -40 and val >= 60:
        return '-40to-60'
    elif val < -60 and val >= 80:
        return '-60to-80'
    elif val < -80 and val >= 100:
        return '-80to-100'
    elif val < 100:
        return 'below100'
    else:
        return 'unknown'

In [7]:
def label_creator(df,freq='1min',shift=-15,shift_column='close'):
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df[~df.index.duplicated(keep='first')]
    label_name = f'label_{shift}_{freq}_{shift_column}'
    df['transition'] = df.shift(shift, freq=freq)[shift_column]
    df[label_name] = df.shift(shift, freq=freq)[shift_column].subtract(df[shift_column]).apply(label_generator)
    return df

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

In [8]:
base_df = label_creator(base_df,freq='1min',shift=-60,shift_column='close')

In [9]:
base_df = label_creator(base_df,freq='1min',shift=-30,shift_column='close')
base_df = label_creator(base_df,freq='1min',shift=-15,shift_column='close')

In [37]:
from sklearn.base import BaseEstimator, TransformerMixin
class PriceAction(BaseEstimator, TransformerMixin):
    def __init__(self, freq='1min',shift=15,shift_column='close'):
        self.freq = freq
        self.shift = shift
        self.shift_column = shift_column
        self.col_name = f'{shift_column}_{freq}_{shift}'
        
    def fit(self, X, y=None):
        return self    # Nothing to do in fit in this scenario
    
    def transform(self, df):
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df = df[~df.index.duplicated(keep='first')]         
        df[self.col_name] = df[self.shift_column].subtract(df.shift(self.shift, freq=self.freq)[self.shift_column])
        df[self.col_name] = df[self.col_name].div(df[self.shift_column])*100
        df[self.col_name] = df[self.col_name].round(3)    
        return df

In [38]:
obj = PriceAction()
obj.transform(base_df)

,open,high,low,close,transition,label_-60_1min_close,label_-30_1min_close,label_-15_1min_close,close_1min_3,close_1min_30,close_1min_15
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,8893.05,20to40,40to60,40to60,NaN,NaN,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,8899.35,below100,20to40,40to60,NaN,NaN,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,8899.55,10to20,40to60,40to60,NaN,NaN,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,8905.55,10to20,20to40,40to60,0.077995,NaN,NaN
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,8906.70,20to40,40to60,60to80,-0.121019,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,NaN,unknown,unknown,unknown,-0.010139,0.107,-0.023
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,NaN,unknown,unknown,unknown,-0.028474,0.097,-0.058
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,NaN,unknown,unknown,unknown,0.001300,0.131,-0.076
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,NaN,unknown,unknown,unknown,-0.002990,0.060,-0.076


In [29]:
def create_price_action_columns(df,freq='1min',shift=15,shift_column='close'):
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df[~df.index.duplicated(keep='first')]
    col_name = f'{shift_column}_{freq}_{shift}'
    df[col_name] = df[shift_column].subtract(df.shift(shift, freq=freq)[shift_column])
    df[col_name] = df[col_name].div(df[shift_column])*100
    df[col_name] = df[col_name].round(3)
    #df[col_name] = df[col_name].apply(label_generator)
    return df
base_df = create_price_action_columns(base_df,freq='1min',shift=30,shift_column='close')

In [36]:
base_df['close_1min_30']


2010-03-02 09:01:00      NaN
2010-03-02 09:02:00      NaN
2010-03-02 09:03:00      NaN
2010-03-02 09:04:00      NaN
2010-03-02 09:05:00      NaN
                       ...  
2022-01-13 15:28:00    0.107
2022-01-13 15:29:00    0.097
2022-01-13 15:30:00    0.131
2022-01-13 15:31:00    0.060
2022-01-13 15:32:00   -0.011
Name: close_1min_30, Length: 1105219, dtype: float64

In [35]:
base_df[base_df['close_1min_30'] == base_df['close_1min_30'].max()]


,open,high,low,close,transition,label_-60_1min_close,label_-30_1min_close,label_-15_1min_close,close_1min_3,close_1min_30
2012-10-05 10:22:00,11655.7,11659.75,11647.55,11650.15,11564.85,below100,below100,below100,0.131329,13.79


In [11]:
base_df['label_-15_1min_close'].value_counts()

below100    571971
20to40      130736
10to20      124392
-10to-20    122283
40to60       52952
unknown      44620
60to80       24285
above100     20223
80to100      12912
-10to10        845
Name: label_-15_1min_close, dtype: int64

In [12]:
base_df['label_-60_1min_close'].value_counts()

below100    470061
unknown     177147
20to40      108958
-10to-20     71103
10to20       70878
40to60       68565
above100     68356
60to80       42199
80to100      27491
-10to10        461
Name: label_-60_1min_close, dtype: int64

In [13]:
pd.set_option('display.max_rows', 500)

In [14]:
base_df[base_df['label_-15_1min_close']=='80to100']

,open,high,low,close,transition,label_-60_1min_close,label_-30_1min_close,label_-15_1min_close
2010-04-20 11:03:00,9376.10,9376.40,9359.20,9363.85,9446.20,above100,80to100,80to100
2010-04-20 11:04:00,9362.30,9369.55,9360.50,9369.55,9457.65,above100,80to100,80to100
2010-04-22 11:06:00,9561.05,9566.45,9561.05,9565.00,9646.75,60to80,60to80,80to100
2010-04-22 11:08:00,9569.20,9571.60,9568.05,9569.40,9649.70,60to80,60to80,80to100
2010-06-17 12:50:00,9461.70,9461.70,9446.30,9447.95,9536.10,above100,80to100,80to100
...,...,...,...,...,...,...,...,...
2022-01-13 10:43:00,38437.75,38464.05,38434.55,38462.45,38546.75,60to80,40to60,80to100
2022-01-13 13:23:00,38431.15,38435.95,38394.75,38401.15,38490.40,40to60,60to80,80to100
2022-01-13 13:28:00,38408.65,38422.15,38405.50,38408.45,38488.65,10to20,60to80,80to100
2022-01-13 13:29:00,38407.45,38409.05,38376.15,38397.90,38487.75,10to20,60to80,80to100


In [15]:
from talib.abstract import *

In [16]:

from signals import signals
def create_techindicators(df):
    sig = signals(df)
    all_methods = []
    a = dict(signals.__dict__)
    for a1,a2 in a.items():
        all_methods.append(a1)
    all_methods = [m1 for m1,m2 in a.items() if m1[:1]!='_']
    methods_run = []
    methods_notrun = []
    for f in all_methods:
        try:
            exec(f'sig.{f}()')
            methods_run.append(f)
        except Exception as e1:
            print(f"Function {f} was unable to run")
            methods_notrun.append(f)
    return sig.df

base_df = create_techindicators(base_df)


Function momentum_AROON was unable to run
Function momentum_AROONOSC was unable to run
Function momentum_MFI was unable to run
Function volumn_AD was unable to run
Function volumn_ADOSC was unable to run
Function volumn_OBV was unable to run
Function pattern_abondoned_baby was unable to run
Function pattern_dark_cloud_cover was unable to run
Function pattern_morning_doji_star was unable to run
Function pattern_morning_star was unable to run
Function stats_linear_reg_slope was unable to run
Function stats_variance was unable to run
Function overlap_bolliner_bands was unable to run
Function overlap_mesa_adaptive_moving_average was unable to run
Function overlap_moving_avg_with_variable was unable to run
Function overlap_parabolic_sar was unable to run
Function overlap_parabolic_sar_ext was unable to run
Function overlap_triple_exp_moving_average_t3 was unable to run


MemoryError: Unable to allocate 725. MiB for an array with shape (86, 1105219) and data type float64

In [ ]:
base_df

,open,high,low,close,transition,label_-60_1min_close,label_-30_1min_close,label_-15_1min_close,momentum_ADX,momentum_ADXR,...,overlap_hilbert_transform,overlap_kaufman_adaptive_moving_avg,overlap_moving_avg,overlap_midpoint,overlap_midpoint_price,overlap_simple_moving_avg,overlap_triangular_moving_average,overlap_weighted_moving_average,close_1min_15,close_1min_3
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,8893.05,20to40,40to60,40to60,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,8899.35,below100,20to40,40to60,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,8899.55,10to20,40to60,40to60,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,8905.55,10to20,20to40,40to60,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,8906.70,20to40,40to60,60to80,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,NaN,unknown,unknown,unknown,11.763497,15.972811,...,38473.204428,38459.224804,38468.136667,38477.225,38475.225,38468.136667,38473.485417,38471.749254,-8.90,1.0
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,NaN,unknown,unknown,unknown,11.976822,16.479083,...,38472.773626,38459.167085,38469.381667,38475.875,38475.225,38469.381667,38474.163333,38471.008179,-22.20,1.0
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,NaN,unknown,unknown,unknown,12.011164,16.867322,...,38472.679648,38459.372223,38471.060000,38473.025,38474.175,38471.060000,38474.471250,38470.770652,-29.25,1.0
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,NaN,unknown,unknown,unknown,12.043054,16.846964,...,38472.396275,38459.472050,38471.833333,38468.325,38471.250,38471.833333,38474.326458,38470.447427,-29.05,1.0


In [27]:
base_df['close_1min_3'].value_counts()

above100    1105219
Name: close_1min_3, dtype: int64